# Title: ...

## 0. Imports

In [2]:
# Notebook similar to: https://www.kaggle.com/code/gracehephzibahm/prediction-of-rent-prices-in-barcelona
# Web scraper with good insights and EDA: https://github.com/agonzalezramos/Idealista-Price-Prediction/tree/main

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd

## 1. Load Data

In [4]:
# Load Data
og_df = pd.read_csv('../data/pisos_clean.csv', sep=',')

C:\Users\pdpau\AppData\Local\Temp\ipykernel_64132\4198264111.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  og_df = pd.read_csv('../data/pisos_clean.csv', sep=',')


## 2. Data Preparation

In [5]:
df = og_df.copy()

df.head(5)

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
0,Casa en calle Urb. Las Mimosas,Ames (San Tome),250.000 €,315 m²,4,793 €/m²,3,31,pisos,a_coruna
1,Piso en Milladoiro,Ames (San Tome),90.000 €,72 m²,2,1.250 €/m²,1,11,pisos,a_coruna
2,Casa en calle La Piedra,A Ortigueira (Cariño),50.000 €,92 m²,2,543 €/m²,1,40,pisos,a_coruna
3,Casa en Perillo,Perillo (Oleiros),359.000 €,244 m²,4,1.471 €/m²,3,39,pisos,a_coruna
4,Chalet en Urbanización Las Mimosas,O Milladoiro (Ames),200.430 €,315 m²,4,636 €/m²,3,24,pisos,a_coruna


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777179 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     777179 non-null  object
 1   location    776886 non-null  object
 2   price       776898 non-null  object
 3   size        776892 non-null  object
 4   rooms       752352 non-null  object
 5   price/m2    744608 non-null  object
 6   bathrooms   752526 non-null  object
 7   Num Photos  776898 non-null  object
 8   type        776898 non-null  object
 9   region      776282 non-null  object
dtypes: object(10)
memory usage: 59.3+ MB


In [7]:
# Drop NaN values
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 719459 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     719459 non-null  object
 1   location    719459 non-null  object
 2   price       719459 non-null  object
 3   size        719459 non-null  object
 4   rooms       719459 non-null  object
 5   price/m2    719459 non-null  object
 6   bathrooms   719459 non-null  object
 7   Num Photos  719459 non-null  object
 8   type        719459 non-null  object
 9   region      719459 non-null  object
dtypes: object(10)
memory usage: 60.4+ MB


## 3. Exploratory Data Analysis

In [8]:
df.tail(5)

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
777170,"Finca rústica en Carrer Bosc, nº 51",Lliçà d'Amunt,1.100.000 €,1.120 m²,10,982 €/m²,1.0,32,fincas_rusticas,valles_oriental_sud
777171,Finca rústica en Els Dominics,Cardedeu,1.500.000 €,520 m²,8,2.884 €/m²,3.0,12,fincas_rusticas,valles_oriental_sud
777176,Finca rústica en La Roca del Vallès,La Roca del Vallès,1.350.000 €,350 m²,5,3.857 €/m²,2.0,9,fincas_rusticas,valles_oriental_sud
777177,"Finca rústica en Bajada Les Valls, nº S/N",La Roca del Vallès,456.000 €,202 m²,5,2.257 €/m²,3.0,30,fincas_rusticas,valles_oriental_sud
777178,Loft en calle Catlunya,Llerona (Les Franqueses del Vallès),155.000 €,87 m²,1,1.781 €/m²,1.0,34,lofts,valles_oriental_sud


### 3.1 Dataset compraventa

In [9]:
# TODO: Explore by location

In [10]:
df['region'].value_counts()

df_counts = df[df['region'].str.contains('sabadell')]

df_counts['region'].value_counts()

region
sabadell    2347
Name: count, dtype: int64

In [11]:
# Filtrar nomes per Catalunya

# value counts de region
df['region'].value_counts()

# Filtrar nomes per Barcelona
df_barna = df[df['region'] == 'barcelona']
df_barna

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
20983,"Piso en Carrer del Bruc, cerca de Carretera de...",Valldaura-Plaça Catalunya (Manresa),85.000 €,87 m²,4,977 €/m²,1,10,pisos,barcelona
20984,"Piso en Carrer de les Orenetes, 37",Viladecavalls,149.900 €,57 m²,2,2.629 €/m²,1,16,pisos,barcelona
20985,"Casa en Carrer de la Creu, cerca de Carrer de ...",Sant Just Desvern,1.095.000 €,371 m²,6,2.951 €/m²,4,46,pisos,barcelona
20986,Piso en Centre-Zona Estació-Passeig de Joan Ma...,Centre-Zona Estació-Passeig de Joan Maragall (...,297.000 €,112 m²,3,2.651 €/m²,2,30,pisos,barcelona
20987,"Piso en Carrer del Retir, 4",Castellar del Vallès,230.000 €,70 m²,3,3.285 €/m²,1,1,pisos,barcelona
...,...,...,...,...,...,...,...,...,...,...
269325,"Loft en Carrer de Francisco Giner, nº 40",La Vila de Gràcia (Distrito Gràcia. Barcelona ...,380.000 €,180 m²,2,2.111 €/m²,2.0,11,lofts,barcelona
269326,"Loft en Carrer de Ramón y Cajal, 58",La Vila de Gràcia (Distrito Gràcia. Barcelona ...,220.000 €,87 m²,3,2.528 €/m²,1.0,8,lofts,barcelona
269327,"Loft en Carrer d'Andrade, nº 218",Sant Martí de Provençals (Distrito Sant Martí....,113.000 €,55 m²,1,2.054 €/m²,1.0,9,lofts,barcelona
269330,Loft en Barcelona,Les Corts (Distrito Les Corts. Barcelona Capital),419.300 €,93 m²,1,4.508 €/m²,1.0,20,lofts,barcelona


In [12]:
# Filtrar si location contiene Manresa
df_barris_barna = df_barna[df_barna['location'].str.contains('Barcelona')]
df_barris_barna['location'].value_counts()

location
La Dreta de l'Eixample (Distrito Eixample. Barcelona Capital)             110
El Raval (Distrito Ciutat Vella. Barcelona Capital)                        86
La Nova Esquerra de l'Eixample (Distrito Eixample. Barcelona Capital)      71
L'Antiga Esquerra de l'Eixample (Distrito Eixample. Barcelona Capital)     70
Sant Gervasi-Galvany (Distrito Sarrià-Sant Gervasi. Barcelona Capital)     69
                                                                         ... 
La Vall d'Hebron (Distrito Horta-Guinardó. Barcelona Capital)               3
Ciutat Meridiana (Distrito Nou Barris. Barcelona Capital)                   2
La Clota (Distrito Horta-Guinardó. Barcelona Capital)                       1
Baró de Viver (Distrito Sant Andreu. Barcelona Capital)                     1
Nou Barris (Barcelona Capital)                                              1
Name: count, Length: 70, dtype: int64

In [ ]:
##### REGIONS CATALANES #####
reg_df = df.copy()

unique_regions = reg_df['region'].unique()

unique_regions_list = list(unique_regions)
unique_regions_list.sort()

unique_regions_list

pd.DataFrame(unique_regions_list).to_csv('../data/unique_regions.csv', index=False)

In [ ]:
reg_df = df.copy()

# Recuperar csv de regions catalanes filtrat manualment
catalan_regions = pd.read_csv('../data/catalan_regions.csv')
catalan_regions_list = list(catalan_regions['0'])
catalan_regions_list

# Filtrar df per regions catalanes
reg_df = reg_df[reg_df['region'].isin(catalan_regions_list)]
reg_df['region'].value_counts()

# Eliminar regions amb menys de 10 entrades
reg_df = reg_df.groupby('region').filter(lambda x: len(x) > 10)
reg_df['region'].value_counts()

region
barcelona               9883
tarragona               8740
girona                  8685
valles_occidental       8117
maresme                 7558
                        ... 
castellnou_de_bages       12
sant_pere_de_torello      12
sant_quirze_safaja        11
vilalba_sasserra          11
muntanyola                11
Name: count, Length: 281, dtype: int64